![ine-divider](https://user-images.githubusercontent.com/7065401/92672068-398e8080-f2ee-11ea-82d6-ad53f7feb5c0.png)
<hr>

# PostgreSQL for Python Developers

## Managing PostgreSQL users and roles

In this project, you should use whatever administration tool(s) you prefer to perform some general database and user management tasks.  

You will need access to a PostgreSQL installation where you have superuser permissions. If you do not have such access elsewhere, installing to your personal workstation is a good idea.  Alternately, you might wish to use a Docker container for a self-contained installation.  See `https://hub.docker.com/_/postgres` for details on that option.  Unless you have a specific need to work with an existing installation, choosing a PostgreSQL version of 12 or higher is best.

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Part 1

**Create a database and users**

* Create a database called `ine_project`.
* Create users/roles `alice`, `bob`, and `clara` with passwords `alicepw`, `bobpw`, and `clarapw`.
* Alice should become invalid on December 31, 2030. The others will remain valid indefinitely.
* None of the users should be permitted to create other users, but Clara is permitted to create databases.
* Test logging in as each of the roles.

If you have configured the server correctly, you should see output similar to the below from the next cells.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>usename</th>
      <th>usesysid</th>
      <th>usecreatedb</th>
      <th>usesuper</th>
      <th>userepl</th>
      <th>usebypassrls</th>
      <th>passwd</th>
      <th>valuntil</th>
      <th>useconfig</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>alice</td>
      <td>36108</td>
      <td>False</td>
      <td>False</td>
      <td>False</td>
      <td>False</td>
      <td>********</td>
      <td>2030-12-31 00:00:00-05:00</td>
      <td>None</td>
    </tr>
    <tr>
      <th>1</th>
      <td>bob</td>
      <td>36109</td>
      <td>False</td>
      <td>False</td>
      <td>False</td>
      <td>False</td>
      <td>********</td>
      <td>NaT</td>
      <td>None</td>
    </tr>
    <tr>
      <th>2</th>
      <td>clara</td>
      <td>36110</td>
      <td>True</td>
      <td>False</td>
      <td>False</td>
      <td>False</td>
      <td>********</td>
      <td>NaT</td>
      <td>None</td>
    </tr>
  </tbody>
</table>

In [36]:
import pandas as pd
import psycopg2
host = 'localhost'  # Adjust host/port as needed for your setup
port = '5432'
conn = psycopg2.connect(database='ine_project', 
                        user='alice', password='alicepw', 
                        host=host, port=port)
cur = conn.cursor()

sql = "SELECT * FROM pg_catalog.pg_user WHERE usename IN ('alice', 'bob', 'clara');"
cur.execute(sql)
pd.DataFrame(cur, columns=[c.name for c in cur.description])

,usename,usesysid,usecreatedb,usesuper,userepl,usebypassrls,passwd,valuntil,useconfig
0,alice,36112,False,False,False,False,********,2030-12-31 00:00:00-05:00,None
1,bob,36113,False,False,False,False,********,NaT,None
2,clara,36114,True,False,False,False,********,NaT,None


**A possible solution**

You may choose to peform this administration using pgAdmin, using some other GUI tool, or using Python calls to your PostgreSQL adapter.  Below are steps within `psql`.

```
$ sudo -u postgres psql
[sudo] password for some_local_user: 
psql (12.5 (Ubuntu 12.5-0ubuntu0.20.04.1))
Type "help" for help.
```
```sql
postgres=# CREATE DATABASE ine_project;
CREATE DATABASE
postgres=# CREATE ROLE Alice VALID UNTIL '2030-12-31' PASSWORD 'alicepw' LOGIN; 
CREATE ROLE
postgres=# CREATE USER Bob PASSWORD 'bobpw'; -- alias USER implies LOGIN
CREATE ROLE
postgres=# CREATE USER Clara PASSWORD 'clarapw' CREATEDB; 
CREATE ROLE
```


![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Part 2

**Create tables and permissions**

For this task, you will create tables and grant your users certain permissions. The purpose of this task is not about data design, so the specific data types and columns you create are not too important.  You may want to use a couple distinctive columns and values within them to check that the behavior is as you intended.

* Create tables `data1`, `data2`, and `data3`.
* Alice should be permitted to perform all actions on all tables.
* Bob should only be able to:
  * Only read from `data1`;
  * Insert new rows into `data2`, but not change or remove existing rows;
  * Modify existing rows in `data3`, but not add or remove any.
* Clara should be remove rows from all tables, but not change or even read them.


If you have configured the server correctly, you should see output similar to the below from the next cells.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>grantee</th>
      <th>table_name</th>
      <th>privilege_type</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>alice</td>
      <td>data1</td>
      <td>INSERT</td>
    </tr>
    <tr>
      <th>1</th>
      <td>alice</td>
      <td>data1</td>
      <td>SELECT</td>
    </tr>
    <tr>
      <th>2</th>
      <td>alice</td>
      <td>data1</td>
      <td>UPDATE</td>
    </tr>
    <tr>
      <th>3</th>
      <td>alice</td>
      <td>data1</td>
      <td>DELETE</td>
    </tr>
    <tr>
      <th>4</th>
      <td>alice</td>
      <td>data1</td>
      <td>TRUNCATE</td>
    </tr>
    <tr>
      <th>5</th>
      <td>alice</td>
      <td>data1</td>
      <td>REFERENCES</td>
    </tr>
    <tr>
      <th>6</th>
      <td>alice</td>
      <td>data1</td>
      <td>TRIGGER</td>
    </tr>
    <tr>
      <th>7</th>
      <td>alice</td>
      <td>data2</td>
      <td>TRIGGER</td>
    </tr>
    <tr>
      <th>8</th>
      <td>alice</td>
      <td>data2</td>
      <td>INSERT</td>
    </tr>
    <tr>
      <th>9</th>
      <td>alice</td>
      <td>data2</td>
      <td>SELECT</td>
    </tr>
    <tr>
      <th>10</th>
      <td>alice</td>
      <td>data2</td>
      <td>UPDATE</td>
    </tr>
    <tr>
      <th>11</th>
      <td>alice</td>
      <td>data2</td>
      <td>DELETE</td>
    </tr>
    <tr>
      <th>12</th>
      <td>alice</td>
      <td>data2</td>
      <td>TRUNCATE</td>
    </tr>
    <tr>
      <th>13</th>
      <td>alice</td>
      <td>data2</td>
      <td>REFERENCES</td>
    </tr>
    <tr>
      <th>14</th>
      <td>alice</td>
      <td>data3</td>
      <td>INSERT</td>
    </tr>
    <tr>
      <th>15</th>
      <td>alice</td>
      <td>data3</td>
      <td>SELECT</td>
    </tr>
    <tr>
      <th>16</th>
      <td>alice</td>
      <td>data3</td>
      <td>UPDATE</td>
    </tr>
    <tr>
      <th>17</th>
      <td>alice</td>
      <td>data3</td>
      <td>DELETE</td>
    </tr>
    <tr>
      <th>18</th>
      <td>alice</td>
      <td>data3</td>
      <td>TRUNCATE</td>
    </tr>
    <tr>
      <th>19</th>
      <td>alice</td>
      <td>data3</td>
      <td>REFERENCES</td>
    </tr>
    <tr>
      <th>20</th>
      <td>alice</td>
      <td>data3</td>
      <td>TRIGGER</td>
    </tr>
  </tbody>
</table>

In [39]:
sql = """
SELECT grantee, table_name, privilege_type 
FROM information_schema.table_privileges 
WHERE grantee NOT IN ('postgres', 'PUBLIC') 
ORDER BY grantee, table_name;
"""
cur.execute(sql)
pd.DataFrame(cur, columns=[c.name for c in cur.description])

,grantee,table_name,privilege_type
0,alice,data1,INSERT
1,alice,data1,SELECT
2,alice,data1,UPDATE
3,alice,data1,DELETE
4,alice,data1,TRUNCATE
5,alice,data1,REFERENCES
6,alice,data1,TRIGGER
7,alice,data2,TRIGGER
8,alice,data2,INSERT
9,alice,data2,SELECT


**A possible solution**

You may choose to peform this administration using pgAdmin, using some other GUI tool, or using Python calls to your PostgreSQL adapter.  Below are steps within `psql`.

```
$ sudo -u postgres psql
[sudo] password for some_local_user: 
psql (12.5 (Ubuntu 12.5-0ubuntu0.20.04.1))
Type "help" for help.

ine=# \c ine_project
You are now connected to database "ine_project" as user "postgres".
```
```sql
ine_project=# CREATE TABLE data1 (id SERIAL, description TEXT);
CREATE TABLE
ine_project=# CREATE TABLE data2 (id SERIAL, description TEXT);
CREATE TABLE
ine_project=# CREATE TABLE data3 (id SERIAL, description TEXT);
CREATE TABLE
ine_project=# GRANT ALL ON data1 TO alice;
GRANT
ine_project=# GRANT ALL ON data2 TO alice;
GRANT
ine_project=# GRANT ALL ON data3 TO alice;
GRANT
ine_project=# GRANT SELECT ON data1 TO bob;
GRANT
ine_project=# GRANT INSERT ON data2 TO bob;
GRANT
ine_project=# GRANT SELECT ON data2 TO bob; -- underspecified in problem
GRANT
ine_project=# GRANT UPDATE ON data3 TO bob;
GRANT
ine_project=# GRANT DELETE ON data1, data2, data3 TO clara;
GRANT
ine_project=# GRANT TRUNCATE ON data1, data2, data3 TO clara; -- truncate a kind of deletion?
GRANT
```


![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)